In [1]:
%pip install libclang langchain_pymupdf4llm

  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached langchain_pymupdf4llm-0.4.1-py3-none-any.whl.metadata (7.0 kB)
  Using cached pymupdf4llm-0.0.27-py3-none-any.whl.metadata (4.8 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pymupdf-1.26.3-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp310-cp310-win_amd64.whl.metadata (3.0 kB)
Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
Using cached langchain_pymupdf4llm-0.4.1-py3-none-any.whl (12 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached pymupdf4llm-0.0.27-py3-none-any.whl (30 kB)
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
Using cached pymupdf-1.26.3-cp39-abi3-win_amd64.whl (18.7 MB)
Using cached requests_toolbelt-1.0.0-py2.p

In [5]:
%pip install pymupdf pymupdf4llm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
from datetime import datetime

In [1]:
import pymupdf
import pymupdf4llm
from pymupdf4llm import to_markdown

#doc = pymupdf.open("test1.pdf")
llama_reader = pymupdf4llm.LlamaMarkdownReader()
doc = llama_reader.load_data("test1.pdf")
#docs = to_markdown(doc)

# print(doc)  # Print the content of the first page
for i, section in enumerate(doc):
    print(f"\n=== Section {i+1} ===\n")
    print(section.text_resource.text)
    print("\n" + "-"*60)

# read the metadata
print(type(doc[0].metadata))


Successfully imported LlamaIndex

=== Section 1 ===

# `02(f)-24-04/2021(P) Kand. 189`

```
                                 16/01/2025 10:59:58

```

**IN THE FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**


**CIVIL APPEAL NO 02(f)-23-04/2021(P)**


**BETWEEN**


**TEOH KIANG HONG** **… APPELLANT**


**AND**


**THEOW SAY KOW @**


**TEOH KIANG SENG, HENRY** **… RESPONDENT**


[In the matter of the Court of Appeal Malaysia

Civil Appeal No. P-02(W)-1182-06/2017


Between


Theow Say Kow @
Teoh Kiang Seng, Henry … Appellant


And


Teoh Kiang Hong … Respondent]

[In the matter of the High Court in Malaya at Pulau Pinang

Suit No. 22-121-2008


Between


Teoh Kiang Hong … Plaintiff


And


Theow Say Kow @
Teoh Kiang Seng, Henry … Defendant]


1

S/N 9FZtw7kygkCReUiJH3PmuQ
**Note : Serial number will be used to verify the originality of this document via eFILING portal




------------------------------------------------------------

=== Section 2 ===

**HEARD TOGETHER**


**IN THE FEDERAL COU

In [2]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

loader = PyMuPDF4LLMLoader("test1.pdf")
docs = loader.load()

for page in docs:
    # page.page_content is a string, so just print it
    print(page.page_content)
    print(type(page))

# `02(f)-24-04/2021(P) Kand. 189`

```
                                 16/01/2025 10:59:58

```

**IN THE FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**


**CIVIL APPEAL NO 02(f)-23-04/2021(P)**


**BETWEEN**


**TEOH KIANG HONG** **… APPELLANT**


**AND**


**THEOW SAY KOW @**


**TEOH KIANG SENG, HENRY** **… RESPONDENT**


[In the matter of the Court of Appeal Malaysia

Civil Appeal No. P-02(W)-1182-06/2017


Between


Theow Say Kow @
Teoh Kiang Seng, Henry … Appellant


And


Teoh Kiang Hong … Respondent]

[In the matter of the High Court in Malaya at Pulau Pinang

Suit No. 22-121-2008


Between


Teoh Kiang Hong … Plaintiff


And


Theow Say Kow @
Teoh Kiang Seng, Henry … Defendant]


1

S/N 9FZtw7kygkCReUiJH3PmuQ
**Note : Serial number will be used to verify the originality of this document via eFILING portal



<class 'langchain_core.documents.base.Document'>
**HEARD TOGETHER**


**IN THE FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**


**CIVIL APPEAL NO 02(f)-24-04/2021(P)**


**BETWEEN**


In [5]:
import re
from datetime import datetime
import ollama
import json
from typing import List, Dict, Optional
from pydantic import BaseModel, Field
# from llama_index.llms.ollama import Ollama
from llama_index.core.output_parsers import PydanticOutputParser

In [24]:
class LegalCaseMetadata(BaseModel):
    case_number: Optional[str]
    court: Optional[str]
    appeal_type: Optional[str]
    decision_date: Optional[str]
    serial_number: Optional[str]
    appellants: List[str] = []
    respondents: List[str] = []
    coram: List[str] = []
    related_cases: List[str] = []
    high_court_cases: List[str] = []
    court_of_appeal_cases: List[str] = []
    
class LegalCaseMetadataUsingLLM(BaseModel):
    """
    Metadata schema for extracting structured information from legal case documents
    using LLMs with LlamaIndex structured prediction.

    Attributes:
        main_issues (List[str]): The key legal issues or questions addressed in the case.
        disposition (str): The outcome of the case (e.g., "appeal allowed", "appeal dismissed").
        type_of_dispute (str): The category of dispute (e.g., "contract", "tort", "family", "land").
        case_result (str): The practical result of the case (e.g., "Plaintiff awarded RM 2 million in damages",
                           "Custody granted to mother", "Defendant declared bankrupt").
    """
    main_issues: List[str] = Field(default_factory=list)
    disposition: str
    type_of_dispute: str
    case_result: str

In [ ]:
import os
import json

# Directory containing JSON files
directory = "./"

# The fields to be remove
fields_to_remove = ["nombor_kes", "pihak_pihak", "kata_kunci"]

for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
        
        # Read the JSON file
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                print(f"❌ Skipping {filename}, not a valid JSON.")
                continue
        
        # Function to remove fields from a dict
        def remove_fields(obj):
            if isinstance(obj, dict):
                for field in fields_to_remove:
                    obj.pop(field, None)  # remove if exists
                for value in obj.values():
                    remove_fields(value)
            elif isinstance(obj, list):
                for item in obj:
                    remove_fields(item)

        # Clean the JSON
        remove_fields(data)

        # Save back the JSON file
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"✅ Updated {filename}")

✅ Updated test.json
✅ Updated test1.json


In [5]:
import os

# Directories
pdf_dir = "../data/raw/legal_cases"
json_dir = "../data/raw/legal_cases/metadata"

# Get base names of PDFs (without extension)
pdf_files = {os.path.splitext(f)[0] for f in os.listdir(pdf_dir) if f.endswith(".pdf")}

# Get base names of JSONs (without extension, and strip "_metadata")
json_files = {
    os.path.splitext(f)[0].replace("_metadata", "")
    for f in os.listdir(json_dir) if f.endswith(".json")
}

# Counter
missing_count = 0

for pdf_name in pdf_files:
    if pdf_name not in json_files:
        print(pdf_name)
        missing_count += 1

print(f"\nTotal missing files: {missing_count}")

WA-24C-71-03_2020_&_WA-24C-93-05_2020_(Mahkamah_Tinggi)
W-02(NCVC)(W)-60-01_2018;_W-02(NCVC)(W)-61-01_2018;_W-02(NCVC)(W)-62-01_2018;_W-02(NCVC)(W)-63-01_2018;_W-02(NCVC)(W)-64-01_2018;_W-02(NCVC)(W)-65-01_2018_(Mahkamah_Rayuan)

Total missing files: 2


In [14]:
import os

# Directories
pdf_dir = "../data/raw/legal_cases"
json_dir = "../data/raw/legal_cases/metadata"

# Threshold (Windows MAX_PATH is 260 by default)
MAX_PATH = 218

def check_long_paths(directory):
    print(f"\n🔎 Checking: {directory}")
    long_files = []
    for root, _, files in os.walk(directory):
        for f in files:
            full_path = os.path.abspath(os.path.join(root, f))
            if len(full_path) >= MAX_PATH:
                long_files.append((len(full_path), full_path))
    return long_files

# Check both dirs
long_pdf = check_long_paths(pdf_dir)
long_json = check_long_paths(json_dir)

# Print results
if long_pdf or long_json:
    print("\n⚠️ Found files with extremely long paths:\n")
    for length, path in sorted(long_pdf + long_json, reverse=True):
        print(f"{length} chars -> {path}")
else:
    print("\n✅ No problematic file paths found (all under 260 chars).")


🔎 Checking: ../data/raw/legal_cases

🔎 Checking: ../data/raw/legal_cases/metadata

✅ No problematic file paths found (all under 260 chars).


In [15]:
import os
import shutil
from math import ceil

# Directories
pdf_dir = "../data/raw/legal_cases"
json_dir = "../data/raw/legal_cases/metadata"

# Parameters
chunk_size = 1000

# Get sorted list of PDFs
pdf_files = sorted([f for f in os.listdir(pdf_dir) if f.endswith(".pdf")])

# Calculate number of folders needed
num_folders = ceil(len(pdf_files) / chunk_size)

print(f"Total PDFs: {len(pdf_files)}")
print(f"Creating {num_folders} folders...")

for i in range(num_folders):
    # Create main folder (folder1, folder2, ...)
    folder_name = f"legal_case_folder{i+1}"
    folder_path = os.path.join(pdf_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Create metadata folder inside
    metadata_path = os.path.join(folder_path, "metadata_folder")
    os.makedirs(metadata_path, exist_ok=True)

    # Get PDFs for this chunk
    start = i * chunk_size
    end = start + chunk_size
    pdf_chunk = pdf_files[start:end]

    for pdf_file in pdf_chunk:
        pdf_base = os.path.splitext(pdf_file)[0]
        
        src_pdf = os.path.join(pdf_dir, pdf_file)
        dst_pdf = os.path.join(folder_path, pdf_file)
        print(f"Moving PDF: {src_pdf} -> {dst_pdf}")
        # debugging
        if not os.path.exists(src_pdf):
            print(f"❌ PDF not found: {src_pdf}")
        # Move PDF
        shutil.move(src_pdf, dst_pdf)

        # Find metadata JSON
        json_file = f"{pdf_base}_metadata.json"
        json_path = os.path.join(json_dir, json_file)

        if os.path.exists(json_path):
            shutil.move(json_path, os.path.join(metadata_path, json_file))
        else:
            print(f"⚠️ No metadata for {pdf_file}")

    print(f"✅ {folder_name} done ({len(pdf_chunk)} PDFs)")

Total PDFs: 15427
Creating 16 folders...
Moving PDF: ../data/raw/legal_cases\(M1)_22-203-2006,_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf -> ../data/raw/legal_cases\legal_case_folder1\(M1)_22-203-2006,_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf
Moving PDF: ../data/raw/legal_cases\(M1)_22-203-2006_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf -> ../data/raw/legal_cases\legal_case_folder1\(M1)_22-203-2006_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf
Moving PDF: ../data/raw/legal_cases\-AA-12BNCvC-5-06_2020_(Mahkamah_Tinggi).pdf -> ../data/raw/legal_cases\legal_case_folder1\-AA-12BNCvC-5-06_2020_(Mahkamah_Tinggi).pdf
Moving PDF: ../data/raw/legal_cases\01(F)-27-09_2016(M)_(Mahkamah_Persekutuan).pdf -> ../data/raw/legal_cases\legal_case_folder1\01(F)-27-09_2016(M)_(Mahkamah_Persekutuan).pdf
Moving PDF: ../data/raw/legal_cases\01(F)-58-10_2017(P)_&_01(F)-59-11_2017(P)_(Mahkamah_Persekutuan).pdf -> ../data/raw/legal_cases\legal_case_folder1\01(F)-58-10_2017(P)_&_01(F)-59-11_2017(P)_(Mahkamah_Persekutuan).pdf
Moving 

In [ ]:
class LegalCaseMetadata(BaseModel):
    case_number: Optional[str]
    court: Optional[str]
    appeal_type: Optional[str]
    decision_date: Optional[str]
    serial_number: Optional[str]
    petitioners: List[str] = []    # Covers: appellants, plaintiffs  
    respondents: List[str] = []    # Covers: respondents, defendants
    coram: List[str] = []
    related_cases: List[str] = []
    legislation_referred: List[str] = []
    other_sources_referred: List[str] = []

    # from LLM
    main_issues: List[str] = []
    disposition: Optional[str]
    type_of_dispute: Optional[str]
    case_result: Optional[str]

    # extra raw fields
    filename: Optional[str]
    case_file_number: Optional[int]

In [15]:
def extract_metadata_using_regex(page_content):
    metadata = {}

    # Case Number
    case_patterns = [
        r"GUAMAN\s*NO\s*:?\s*([\w\(\)\-\/\s]+)",  
        r"RAYUAN SIVIL NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",  
        r"GUAMAN SIVIL NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",  
        r"CIVIL SUIT NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",
        r"SUIT NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",                  
        r"CIVIL APPEAL NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",
        r"APPEAL NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",
        r"NO\.?\s*:?\s*([\w\(\)\-\/\s]+)",
    ]

    for pattern in case_patterns:
        match = re.search(pattern, page_content, re.IGNORECASE)
        if match:
            metadata["case_number"] = match.group(1).strip()
            break

    # Court
    court_patterns = [
        # Malaysian court format - capture court type before location
        r"DALAM\s+(MAHKAMAH\s+[A-Z\s]+?)(?:\s+DI\s+[A-Z\s]+|\s+DALAM\s+NEGERI|\n|$)",
        # English format - specific court keywords only  
        r"IN THE\s+((?:FEDERAL COURT|HIGH COURT|COURT OF APPEAL|SESSIONS COURT|MAGISTRATES?[\'\s]*COURT)[^\n]*?)(?:\s+AT\s+[A-Z\s]+)?(?:\n|$)",
        # Alternative English format
        r"IN THE\s+(.*?COURT.*?)(?:\s+AT\s+|\n|$)",
    ]
    
    metadata["court"] = None
    for pattern in court_patterns:
        court_match = re.search(pattern, page_content, re.IGNORECASE)
        if court_match:
            court_text = court_match.group(1).strip()
            # Additional validation - must contain "COURT" or "MAHKAMAH"
            if "COURT" in court_text.upper() or "MAHKAMAH" in court_text.upper():
                metadata["court"] = court_text
                break

    # Appeal Type, Regex: matches "Civil Appeal", "Appeal (Civil)", "Rayuan Sivil", any capitalization
    pattern = r"(Civil\s+Appeal|Appeal\s*\(Civil\)|Rayuan\s+Sivil)"

    if re.search(pattern, page_content, re.IGNORECASE):
        metadata["appeal_type"] = "Civil Appeal"
    else:
        metadata["appeal_type"] = "Unknown"

    # Decision Date
    date_match = re.search(r"(\d{2}/\d{2}/\d{4})", page_content)
    if date_match:
        raw_date = date_match.group(1)
        parsed_date = datetime.strptime(raw_date, "%d/%m/%Y")
        metadata["decision_date"] = parsed_date.strftime("%Y-%m-%d")
    else:
        metadata["decision_date"] = None
        
    # Serial Number
    serial_match = re.search(r"(?:SIN|S/N)\s+([a-zA-Z0-9]{15,25})", page_content)
    metadata["serial_number"] = serial_match.group(1) if serial_match else None
    
    # UNIVERSAL registration number pattern for all party types
    # Matches: [No. K/P:123], [No. Pendaftaran:456], [No. Syarikat:789], etc.
    registration_pattern = r"(?:\s+\[No\.\s+(?:K/P|Pendaftaran|Perniagaan|Syarikat):[^\]]+\])?"

    # Appellant patterns (for appeals)
    appellant_patterns = [
        r"BETWEEN\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+APPELLANT?\b",
        r"BETWEEN\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+APPELLANTS?\b",
        r"ANTARA\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+PERAYU(?:-PERAYU)?\b",
    ]

    # Respondent patterns (for appeals)
    respondent_patterns = [
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+RESPONDENT?\b",
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+RESPONDENTS?\b",
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+RESPONDEN(?:-RESPONDEN)?\b",
    ]

    # Plaintiff patterns (for original suits)
    plaintiff_patterns = [
        r"BETWEEN\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+PLAINTIFF?\b",
        r"BETWEEN\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+PLAINTIFFS?\b",
        r"ANTARA\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+PLAINTIF(?:-PLAINTIF)?\b",
        # Simple pattern without dots: ANTARA [Name] [Registration] PLAINTIF
        r"ANTARA\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+PLAINTIF\b",
    ]

    # Defendant patterns (for original suits)
    defendant_patterns = [
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+DEFENDANT?\b",
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+DEFENDANTS?\b",
        r"(?:^|\n)\s*(?:AND|DAN)\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+[…\.]{3,}\s+DEFENDAN(?:-DEFENDAN)?\b",
        # Simple pattern without dots: DAN [Name] [Registration] DEFENDAN
        r"(?:^|\n)\s*DAN\s+([A-Z][A-Z\s]{10,50}?)" + registration_pattern + r"\s+DEFENDAN\b",
    ]

    # Extract parties with controlled matching
    first_parties = []
    second_parties = []
    
    # Try appellant patterns first (for appeals)
    for pattern in appellant_patterns:
        matches = re.findall(pattern, page_content, re.IGNORECASE | re.MULTILINE)
        if matches:
            first_parties.extend(matches)
            break
    
    # If no appellants, try plaintiff patterns (for original suits)
    if not first_parties:
        for pattern in plaintiff_patterns:
            matches = re.findall(pattern, page_content, re.IGNORECASE | re.MULTILINE)
            if matches:
                first_parties.extend(matches)
                break

    # Try respondent patterns first (for appeals)
    for pattern in respondent_patterns:
        matches = re.findall(pattern, page_content, re.IGNORECASE | re.MULTILINE)
        if matches:
            second_parties.extend(matches)
            break
    
    # If no respondents, try defendant patterns (for original suits)
    if not second_parties:
        for pattern in defendant_patterns:
            matches = re.findall(pattern, page_content, re.IGNORECASE | re.MULTILINE)
            if matches:
                second_parties.extend(matches)
                break

    # Enhanced name cleaning and validation
    def clean_and_validate_party_name(name):
        # Remove ALL types of registration numbers
        cleaned = re.sub(r'\[No\.\s+(?:K/P|Pendaftaran|Syarikat):[^\]]+\]', '', name)
        # Normalize whitespace
        cleaned = re.sub(r'\s+', ' ', cleaned.strip())
        
        # Validate: must be proper name (letters and spaces only, reasonable length)
        if (len(cleaned) > 5 and len(cleaned) < 80 and 
            re.match(r'^[A-Z][A-Z\s&.,()-]+$', cleaned) and
            not any(word in cleaned.lower() for word in ['seksyen', 'akta', 'plaintif', 'defendan', 'dan', 'yang', 'untuk', 'dalam', 'dengan', 'pada', 'atau'])):
            return cleaned
        return None

    # Store validated parties
    if first_parties:
        cleaned_first = [clean_and_validate_party_name(name) for name in first_parties]
        metadata["petitioners"] = [name for name in cleaned_first if name is not None]
    else:
        metadata["petitioners"] = []
    
    if second_parties:
        cleaned_second = [clean_and_validate_party_name(name) for name in second_parties]
        metadata["respondents"] = [name for name in cleaned_second if name is not None]
    else:
        metadata["respondents"] = []

    # Judges (Coram)
    coram_match = re.search(r"CORAM:\s*(.*?)\n\n", page_content, re.IGNORECASE | re.DOTALL)
    if coram_match:
        judges_raw = coram_match.group(1).split('\n')
        metadata["coram"] = [judge.strip() for judge in judges_raw if judge.strip()]
    else:
        metadata["coram"] = []
        
    def extract_references_by_keywords(page_content):
        """Extract references based on specific keywords and sections"""
        
        # Initialize lists
        cases_referred = []
        legislation_referred = []
        other_sources_referred = []
        
        # Define keyword patterns for each category
        cases_keywords = [
            "Cases referred:",
            "Cases cited",
            "Cases referred to:",
        ]
        
        legislation_keywords = [
            "Legislation referred:",
            "Legislation referred to:",
            "Legislation cited",
            "Acts/Laws referred:",
            "Acts/Laws referred to:",
            "Acts/Laws cited",
            "ActslLaws referred:",
            "ActslLaws referred to:",
            "ActslLaws cited",
        ]
        
        other_sources_keywords = [
            "Other sources referred to:",
            "Other sources referred:",
            "Other sources cited",
            "Articles referred to:",
            "Articles referred:",
            "Articles cited",
        ]
        
        # Create combined patterns with case-insensitive matching
        all_keywords = cases_keywords + legislation_keywords + other_sources_keywords
        
        # Find all keyword positions
        keyword_positions = []
        for keyword in all_keywords:
            for match in re.finditer(re.escape(keyword), page_content, re.IGNORECASE):
                keyword_positions.append((match.start(), match.end(), keyword.lower()))
        
        # Sort by position
        keyword_positions.sort()
        
        # Extract content between keywords
        for i, (start, end, keyword) in enumerate(keyword_positions):
            # Find the end position (next keyword or end of content)
            if i + 1 < len(keyword_positions):
                content_end = keyword_positions[i + 1][0]
            else:
                content_end = len(page_content)
            
            # Extract the content
            content = page_content[end:content_end].strip()
            
            # Split into lines and clean up
            lines = [line.strip() for line in content.split('\n') if line.strip()]
            
            # 🚨 NEW: Filter out lines containing serial numbers
            filtered_lines = []
            for line in lines:
                # Check if line contains serial number pattern
                if not re.search(r"(?:SIN|S/N)\s+[a-zA-Z0-9]{15,25}", line, re.IGNORECASE):
                    filtered_lines.append(line)
            
            # Categorize based on keyword
            if any(k in keyword for k in ["cases referred", "cases cited"]):
                cases_referred.extend(filtered_lines)
            elif any(k in keyword for k in ["legislation", "acts/laws", "actsllaws"]):
                legislation_referred.extend(filtered_lines)
            elif any(k in keyword for k in ["other sources", "articles"]):
                other_sources_referred.extend(filtered_lines)
        
        return cases_referred, legislation_referred, other_sources_referred

    # Use the new extraction function
    cases_referred, legislation_referred, other_sources_referred = extract_references_by_keywords(page_content)
    
    # Clean up references - remove duplicates and empty entries
    def clean_references(ref_list):
        """Remove duplicates and clean up references"""
        cleaned = []
        seen = set()
        for ref in ref_list:
            ref_clean = ref.strip()
            # Remove bullet points and list markers
            ref_clean = re.sub(r'^[>➤▶•\-\d+\)\.]?\s*', '', ref_clean)
            
            if ref_clean and ref_clean not in seen and len(ref_clean) > 5:
                cleaned.append(ref_clean)
                seen.add(ref_clean)
        return cleaned

    # Set the metadata with proper field names
    metadata["related_cases"] = clean_references(cases_referred)
    metadata["legislation_referred"] = clean_references(legislation_referred)
    metadata["other_sources_referred"] = clean_references(other_sources_referred)

    return metadata

In [ ]:
# import pymupdf

# doc = pymupdf.open("test.pdf")
# for page_num in range(len(doc)):
#     images = doc[page_num].get_images(full=True)
#     for img in images:
#         xref = img[0]
#         pix = pymupdf.Pixmap(doc, xref)
#         print(f"Image {xref}: {pix.width}x{pix.height} pixels")

In [31]:
import json
from collections import OrderedDict

In [ ]:
# # Keys you want to check
# keys_to_check = ["nombor_kes", "tarikh_keputusan"]

# file_path = "test.json"

# with open("test.json", "r", encoding="utf-8") as f:
#     data = json.load(f)
    
# # Extract court name from nombor_kes (anything inside parentheses at the end)
# nombor_kes = data.get("nombor_kes", "").strip()
# court_match = re.search(r"\(([^)]+)\)\s*$", nombor_kes)

# court_name = None
# if court_match:
#     court_name = court_match.group(1)
#     # Remove the court from nombor_kes
#     nombor_kes = nombor_kes[:court_match.start()].strip()
#     data["nombor_kes"] = nombor_kes

# if court_name:
#     # Rebuild dict so "court" is inserted right after "nombor_kes"
#     new_data = OrderedDict()
#     for k, v in data.items():
#         new_data[k] = v
#         if k == "nombor_kes":
#             new_data["court"] = court_name

#     # Save back to file
#     with open(file_path, "w", encoding="utf-8") as f:
#         json.dump(new_data, f, indent=2, ensure_ascii=False)

#     print(f"✅ Extracted court: {court_name}, cleaned nombor_kes: {nombor_kes}")
# else:
#     print("⚠️ No court name found in nombor_kes")

✅ Extracted court: Mahkamah Persekutuan, cleaned nombor_kes: 02(f)-2-01/2023(W)


In [7]:
# ocr pipelines
import easyocr
from pdf2image import convert_from_path
import numpy as np

def ocr_pdf_to_text(pdf_path, dpi=300, langs=['en', 'ms'], gpu=True):
    """
    Perform OCR on a PDF and return a list of text results for each page.

    Args:
        pdf_path (str): Path to the PDF file.
        dpi (int): DPI for PDF to image conversion.
        langs (list): Languages for OCR.
        gpu (bool): Use GPU for OCR.

    Returns:
        list: List of OCR results per page.
    """
    pages = convert_from_path(pdf_path, dpi=dpi)
    reader = easyocr.Reader(langs, gpu=gpu)
    results = []
    for page in pages:
        page_np = np.array(page)
        result = reader.readtext(page_np, batch_size=2, detail=0, paragraph=True)
        results.append(result)
    return results

In [8]:
results = ocr_pdf_to_text("test2.pdf")

In [16]:
results

[['WA-12BNCvC-112-08/2023',
  'Kand. 33',
  '24/10/2024 15 :40 : 07',
  'DALAM MAHKAMAH TINGGI MALAYA DI KUALA LUMPUR DALAM WILAYAH PERSEKUTUAN MALAYSIA (BIDANG KUASA RAYUAN) RAYUAN SIVIL NO : WA-I2BNCVC-112-08/2023',
  'ANTARA',
  '1 . NG CHIN TUAN (No. KIP: 840524-14-5975) 2. NG LEE WERN (No. KIP: 880405-14-5418) 3 NG CHIN KEONG (No. KIP: 800318-14-5353) 4 NG LEE MIN (No. KIP: 811119-14-5952) 5. NG TECK HUAT (No. KIP: 490314-08-5401) 6 TEH KIM CHOW (No KIP: 500917-08-5474)',
  'Berniaga di bawah nama dan gaya SYARIKAT SIN HIN ENGINEERING (No. Perniagaan:  000358977-X)',
  'PERAYU- PERAYU',
  'DAN',
  'MULTIPLE BUILDERS SDN BHD (No. Syarikat:   11182-W)',
  'RESPONDEN',
  '[Dalam Perkara Guaman Sivil No.: WA-BS2NCVC-165-04/2022 Dalam Mahkamah Sesyen di Kuala Lumpur',
  'Antara',
  'MULTIPLE BUILDERS SDN BHD (No. Syarikat:   11182-W)',
  'PLAINTIF',
  'SIN 7FguAEsSOin53tcRgIWTA ** Note Serial number will be used to verify the originality of this document via eFILING portal'],
 ['DAN',


In [18]:
all_pages_text = "\n\n".join(["\n".join(page_text_list) for page_text_list in results])
metadata = extract_metadata_using_regex(all_pages_text)
metadata

{'case_number': 'WA-I2BNCVC-112-08/2023\nANTARA\n1',
 'court': 'MAHKAMAH TINGGI MALAYA',
 'appeal_type': 'Civil Appeal',
 'decision_date': '2024-10-24',
 'serial_number': '7FguAEsSOin53tcRgIWTA',
 'petitioners': [],
 'respondents': [],
 'coram': [],
 'related_cases': ['Gan Yook Chin (P) & Anor v Lee Ing Chin CLJ 309 FC',
  'Lee Teck Seng & Ors (2004) 4',
  "Ong Leong Chiou & Anor v Keller (M) Sdn Bhd & Ors (2021) 3 MLJ 622 FC Tengku Dato' Ibrahim Petra bin Tengku Indra Petra V Petra Perdana Bhd (2018) 2 MLJ 177 FC",
  'Ng Hoo Kui & Anor v Wendy Tan Lee Peng (2020) 12 MLJ 67 FC Sarmiina Sdn Bhd v Gerry Ho & Ors (2023) 5 MLRA 159 FC',
  'Sejati Education Sdn Bhd v S3M Development (Sabah) Sdn Bhd (2016) 6 CLJ 710 CA',
  'FKJV (M) Sdn Bhd v Mode Circle Bhd (2012) MLJU 751 HC',
  'Tieh Boon Tuck v Evonne Lee Pei Chen & Anor (2014) 1 MLJ 882 CA Cipta Cermat Sdn Bhd v Perbandaran Kemajuan Negeri Kedah (2007) 2 MLJ 746 CA',
  'Lai Hee Sang v Pun Hai Chin (2016) 6 MLJ 434 CA',
  'Bonham Carter 

In [24]:
metadata = extract_metadata_using_regex(docs[0].page_content)
metadata

{'case_number': '02(f)-23-04/2021(P)',
 'court': 'FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**',
 'appeal_type': 'Civil Appeal',
 'decision_date': '2025-01-16',
 'serial_number': '9FZtw7kygkCReUiJH3PmuQ',
 'petitioners': [],
 'respondents': [],
 'coram': [],
 'related_cases': [],
 'high_court_cases': ['22-121-2008'],
 'court_of_appeal_cases': []}

In [ ]:
# Use the PydanticOutputParser so the LLM fills our schema
parser = PydanticOutputParser(output_cls=LegalCaseMetadataUsingLLM)

In [ ]:
import json
import os
import re
from collections import OrderedDict

# Key mapping: old → new
key_mapping = {
    "nombor_kes": "case_number",
    "tarikh_keputusan": "decision_date",
    "pihak_pihak": "petitioners_and_respondents",
    "hakim_majistret": "coram"
}

def rename_keys(obj):
    """Recursively rename keys in dicts/lists."""
    if isinstance(obj, dict):
        new_obj = {}
        for k, v in obj.items():
            new_key = key_mapping.get(k, k)  # rename if mapping exists
            new_obj[new_key] = rename_keys(v)
        return new_obj
    elif isinstance(obj, list):
        return [rename_keys(i) for i in obj]
    else:
        return obj
metadata = {
    'case_number': '02(f)-23-04/2021(P)',
    'court': 'FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**',
    'appeal_type': 'Civil Appeal',
    'decision_date': '2025-01-16',
    'serial_number': '9FZtw7kygkCReUiJH3PmuQ',
    'petitioners': [],
    'respondents': [],
    'coram': [],
    'related_cases': [],
    'high_court_cases': ['22-121-2008'],
    'court_of_appeal_cases': []
}
  
# Fields we need to check for emptiness
check_fields = ["petitioners_and_respondents", "decision_date", "coram"]


# Exclude these when appending metadata at the end
exclude_keys = {"case_number", "decision_date", "petitioners_and_respondents", "coram"}

# Loop through 16 folders
for i in range(1, 17):
    json_dir = f"../data/raw/legal_cases/legal_case_folder{i}/metadata_folder"

    if not os.path.exists(json_dir):
        print(f"⚠️ Skipping {json_dir}, does not exist.")
        continue

    for filename in os.listdir(json_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(json_dir, filename)

            # Load JSON
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
            except json.JSONDecodeError:
                print(f"❌ Skipping invalid JSON: {file_path}")
                continue

            # Step 1: Rename keys
            data = rename_keys(data)

            # Step 2: Extract "court" from case_number
            case_number = data.get("case_number", "").strip()
            court_match = re.search(r"\(([^)]+)\)\s*$", case_number)

            court_name = None
            if court_match:
                court_name = court_match.group(1)
                # remove court from case_number string
                case_number = case_number[:court_match.start()].strip()
                data["case_number"] = case_number

            # Step 3: Ensure "court" right after "case_number"
            ordered_data = OrderedDict()
            for k, v in data.items():
                ordered_data[k] = v
                if k == "case_number" and court_name:
                    ordered_data["court"] = court_name
                
                        # Step 4: Check empty fields and fill from metadata
            for field, meta_key in check_fields.items():
                if field not in ordered_data or not ordered_data[field]:
                    if isinstance(meta_key, list):  
                        # Special case for petitioners_and_respondents
                        petitioners = metadata.get("petitioners", [])
                        respondents = metadata.get("respondents", [])
                        if petitioners or respondents:
                            ordered_data[field] = {
                                "petitioners": petitioners,
                                "respondents": respondents
                            }
                    else:
                        if metadata.get(meta_key):
                            ordered_data[field] = metadata[meta_key]

            # Step 5: Append remaining metadata fields at the end (excluding duplicates)
            for k, v in metadata.items():
                if k not in exclude_keys and k not in ordered_data:
                    ordered_data[k] = v

            # Save updated JSON
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            print(f"✅ Updated {filename} in {json_dir} | Court: {court_name}")

✅ Updated test1.json


In [19]:
import json
import re
from collections import OrderedDict

# Key mapping: old → new
key_mapping = {
    "nombor_kes": "case_number",
    "tarikh_keputusan": "decision_date",
    "pihak_pihak": "petitioners_and_respondents",
    "hakim_majistret": "coram"
}

def rename_keys(obj):
    """Recursively rename keys in dicts/lists."""
    if isinstance(obj, dict):
        new_obj = {}
        for k, v in obj.items():
            new_key = key_mapping.get(k, k)
            new_obj[new_key] = rename_keys(v)
        return new_obj
    elif isinstance(obj, list):
        return [rename_keys(i) for i in obj]
    else:
        return obj

# Example metadata object from PDF extraction
# metadata = {
#     'case_number': '02(f)-23-04/2021(P)',
#     'court': 'FEDERAL COURT OF MALAYSIA AT PUTRAJAYA**',
#     'appeal_type': 'Civil Appeal',
#     'decision_date': '2025-01-16',
#     'serial_number': '9FZtw7kygkCReUiJH3PmuQ',
#     'petitioners': [],
#     'respondents': [],
#     'coram': [],
#     'related_cases': [],
#     'high_court_cases': ['22-121-2008'],
#     'court_of_appeal_cases': []
# }

# Fields we need to check for emptiness
check_fields = {
    "petitioners_and_respondents": ["petitioners", "respondents"],
    "decision_date": "decision_date",
    "coram": "coram"
}

exclude_keys = {"case_number", "decision_date", "petitioners", "respondents"}

# 🟢 Change this to the JSON file you want to test
file_path = "./test_test2pdf.json"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
except json.JSONDecodeError:
    print(f"❌ Invalid JSON: {file_path}")
    exit()

# Step 1: Rename keys
data = rename_keys(data)

# Step 2: Extract "court" from case_number
case_number = data.get("case_number", "").strip()
court_match = re.search(r"\(([^)]+)\)\s*$", case_number)

court_name = None
if court_match:
    court_name = court_match.group(1)
    case_number = case_number[:court_match.start()].strip()
    data["case_number"] = case_number

# # Step 3: Ensure "court" right after "case_number"
ordered_data = OrderedDict()
for k, v in data.items():
    ordered_data[k] = v
#     if k == "case_number" and court_name:
#         ordered_data["court"] = court_name

# Step 4: Fill empty fields from metadata
for field, meta_key in check_fields.items():
    if field not in ordered_data or not ordered_data[field]:
        if isinstance(meta_key, list):  
            petitioners = metadata.get("petitioners", [])
            respondents = metadata.get("respondents", [])
            if petitioners or respondents:  # only add if non-empty
                ordered_data[field] = {
                    "petitioners": petitioners,
                    "respondents": respondents
                }
        else:
            if metadata.get(meta_key):
                ordered_data[field] = metadata[meta_key]

# Step 5: Append extra metadata
for k, v in metadata.items():
    if k not in exclude_keys and k not in ordered_data:
        ordered_data[k] = v

# Save updated JSON
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(ordered_data, f, ensure_ascii=False, indent=4)

print(f"✅ Updated {file_path} | Court: {court_name}")

✅ Updated ./test_test2pdf.json | Court: Mahkamah Tinggi


In [ ]:
def clean_json_output(raw_text):
    # Remove triple backticks if present
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_text.strip(), flags=re.MULTILINE)
    return cleaned.strip()

def extract_metadata_using_LLM(text):
    prompt = f"""
    Text:
    \"\"\"
    {text}
    \"\"\"
    """

    response = ollama.generate(
        model="sensitive_information_detector",
        prompt=prompt
    )

    # Clean raw model output
    raw_output = response['response']
    cleaned_output = clean_json_output(raw_output)
    print("Cleaned JSON output:", cleaned_output)

    # Try parsing
    try:
        result = json.loads(cleaned_output)
        redacted_text = result.get("redacted_text", None)
    except Exception as e:
        print("Error parsing cleaned JSON:", e)
        redacted_text = None

    return redacted_text

In [20]:
# Removing information such as identifcation numbers, phone numbers, emails, passport numbers, and dates
PATTERNS = [
    # ---------- SSM ----------
    # Old SSM: 1-7 digits + mandatory -A/-U/-X ...  (do this first!)
    r"\b\d{6,7}-[A-Z]\b",
    # Old SSM: 1-7 digits without suffix
    r"\b\d{6,7}\b",
    # New SSM: 4-digit year + 2-digit entity + 6-digit seq (total 12 digits)
    r"\b(20\d{2})(0[1-6]|11|21)\d{6}\b",

    # ---------- Other IDs ----------
    r"\b\d{6}-\d{2}-\d{4}\b",                        # IC
    r"\b[A-Z]{1,2}\d{6,7}\b",                   # Passport
    r"\b\d{2}/\d{2}/\d{4}\b|\b\d{4}-\d{2}-\d{2}\b",  # Date

    # ---------- Contact ----------
    # e-mail
    r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
    # phone – must contain a hyphen/space between blocks
    # and must NOT be immediately followed by -A/-Z (negative look-ahead)
    r"(?:\+?60\d{1,2}[-\s]?)?\d{2,3}[-\s]\d{4}(?!-[A-Z])",
]

COMPILED = re.compile("|".join(f"(?:{p})" for p in PATTERNS))

def regex_redact_sensitive_info(text: str) -> str:
    return COMPILED.sub("[REDACTED]", text)

In [21]:
# Example usage
text = (
    "Old: 1234567-A, 1-A, 1234567, 123456-U. "
    "New: 201901000001, 202006123456. "
    "IC: 900101-14-5678, Email: test@gmail.com "
    "A1234567"
)
print(regex_redact_sensitive_info(text))

Old: [REDACTED], 1-A, [REDACTED], [REDACTED]. New: [REDACTED], [REDACTED]. IC: [REDACTED]-[REDACTED], Email: [REDACTED] [REDACTED]


In [10]:
def clean_json_output(raw_text):
    # Remove triple backticks if present
    cleaned = re.sub(r"^```json\s*|\s*```$", "", raw_text.strip(), flags=re.MULTILINE)
    return cleaned.strip()

def llm_redact_sensitive_info(text):
    prompt = f"""
    Text:
    \"\"\"
    {text}
    \"\"\"
    """

    response = ollama.generate(
        model="sensitive_information_detector",
        prompt=prompt
    )

    # Clean raw model output
    raw_output = response['response']
    cleaned_output = clean_json_output(raw_output)

    # Try parsing
    try:
        result = json.loads(cleaned_output)
        redacted_text = result.get("redacted_text", None)
    except Exception as e:
        print("Error parsing cleaned JSON:", e)
        redacted_text = None

    return redacted_text

In [15]:
def redact_sensitive_info(text):
    regex_hits = regex_redact_sensitive_info(text)
    #llm_hits = llm_redact_sensitive_info(regex_hits)

    return regex_hits

In [17]:
# Example
text = """John Doe (IC: 900101-14-5678) agreed to a confidential settlement with ABC Corp."""
result = redact_sensitive_info(text)
print(json.dumps(result, indent=2))
print(result)

"John Doe (IC: [REDACTED]-[REDACTED]) agreed to a confidential settlement with ABC Corp."
John Doe (IC: [REDACTED]-[REDACTED]) agreed to a confidential settlement with ABC Corp.


In [26]:
def update_json_with_pdf_metadata(file_path, pdf_extracted_metadata, save_file=True):
    """
    Update a JSON file with metadata extracted from PDF
    
    Args:
        file_path (str): Path to the JSON file to update
        pdf_extracted_metadata (dict): Metadata extracted from PDF using extract_metadata_using_regex()
        save_file (bool): Whether to save the file or just return the updated data
    
    Returns:
        dict: Updated JSON data with merged metadata
    """
    
    # Key mapping: old → new
    key_mapping = {
        "nombor_kes": "case_number",
        "tarikh_keputusan": "decision_date",
        "pihak_pihak": "petitioners_and_respondents",
        "hakim_majistret": "coram"
    }

    def rename_keys(obj):
        """Recursively rename keys in dicts/lists."""
        if isinstance(obj, dict):
            new_obj = {}
            for k, v in obj.items():
                new_key = key_mapping.get(k, k)
                new_obj[new_key] = rename_keys(v)
            return new_obj
        elif isinstance(obj, list):
            return [rename_keys(i) for i in obj]
        else:
            return obj

    # Fields we need to check for emptiness
    check_fields = {
        "petitioners_and_respondents": ["petitioners", "respondents"],
        "decision_date": "decision_date",
        "coram": "coram"
    }

    exclude_keys = {"case_number", "decision_date", "petitioners", "respondents"}

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except json.JSONDecodeError:
        print(f"❌ Invalid JSON: {file_path}")
        return None
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return None

    # Step 1: Rename keys
    data = rename_keys(data)

    # Step 2: Extract "court" from case_number
    case_number = data.get("case_number", "").strip()
    court_match = re.search(r"\(([^)]+)\)\s*$", case_number)

    court_name = None
    if court_match:
        court_name = court_match.group(1)
        case_number = case_number[:court_match.start()].strip()
        data["case_number"] = case_number

    # Step 3: Ensure "court" right after "case_number"
    ordered_data = OrderedDict()
    for k, v in data.items():
        ordered_data[k] = v
        if k == "case_number" and court_name:
            ordered_data["court"] = court_name

    # Step 4: Fill empty fields from PDF metadata
    # To merge petitioners and respondents into one string
    if not ordered_data.get("petitioners_and_respondents"):
        petitioners = pdf_extracted_metadata.get("petitioners")
        respondents = pdf_extracted_metadata.get("respondents")

        if petitioners or respondents:
            parts = []
            if petitioners:
                parts.append(f"petitioners: {petitioners}")
            if respondents:
                parts.append(f"respondents: {respondents}")
            ordered_data["petitioners_and_respondents"] = " and ".join(parts)

    # Step 5: Add decision_date and coram if missing
    if not ordered_data.get("decision_date") and pdf_extracted_metadata.get("decision_date"):
        ordered_data["decision_date"] = pdf_extracted_metadata["decision_date"]

    if not ordered_data.get("coram") and pdf_extracted_metadata.get("coram"):
        ordered_data["coram"] = pdf_extracted_metadata["coram"]

    # Step 6: Append extra metadata
    for k, v in pdf_extracted_metadata.items():
        if k not in exclude_keys and k not in ordered_data:
            ordered_data[k] = v

    # Save updated JSON if requested
    if save_file:
        try:
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(ordered_data, f, ensure_ascii=False, indent=4)
            print(f"✅ Updated {file_path} | Court: {court_name}")
        except Exception as e:
            print(f"❌ Error saving file: {e}")
            return None

    return ordered_data

In [ ]:
def batch_update_json_files_with_pdf_metadata(pdf_extracted_metadata, 
                                           json_dirs=None, 
                                           specific_file=None):
    """
    Batch update multiple JSON files or a specific file with PDF metadata
    
    Args:
        pdf_extracted_metadata (dict): Metadata extracted from PDF
        json_dirs (list): List of directories containing JSON files to update
        specific_file (str): Path to a specific JSON file to update
    
    Returns:
        dict: Summary of updates performed
    """
    
    update_summary = {
        "success_count": 0,
        "error_count": 0,
        "files_processed": [],
        "errors": []
    }
    
    if specific_file:
        # Update single file
        try:
            result = update_json_with_pdf_metadata(specific_file, pdf_extracted_metadata)
            if result:
                update_summary["success_count"] += 1
                update_summary["files_processed"].append(specific_file)
            else:
                update_summary["error_count"] += 1
                update_summary["errors"].append(f"Failed to update {specific_file}")
        except Exception as e:
            update_summary["error_count"] += 1
            update_summary["errors"].append(f"Error with {specific_file}: {e}")
    
    elif json_dirs:
        # Update multiple directories
        for json_dir in json_dirs:
            if not os.path.exists(json_dir):
                print(f"⚠️ Skipping {json_dir}, does not exist.")
                update_summary["errors"].append(f"Directory not found: {json_dir}")
                continue

            for filename in os.listdir(json_dir):
                if filename.endswith(".json"):
                    file_path = os.path.join(json_dir, filename)
                    
                    try:
                        result = update_json_with_pdf_metadata(file_path, pdf_extracted_metadata)
                        if result:
                            update_summary["success_count"] += 1
                            update_summary["files_processed"].append(file_path)
                        else:
                            update_summary["error_count"] += 1
                            update_summary["errors"].append(f"Failed to update {file_path}")
                    except Exception as e:
                        update_summary["error_count"] += 1
                        update_summary["errors"].append(f"Error with {file_path}: {e}")
    
    # Print summary
    print(f"\n=== UPDATE SUMMARY ===")
    print(f"✅ Successfully updated: {update_summary['success_count']} files")
    print(f"❌ Errors: {update_summary['error_count']} files")
    
    if update_summary["errors"]:
        print(f"\n⚠️ Errors encountered:")
        for error in update_summary["errors"][:5]:  # Show first 5 errors
            print(f"  - {error}")
        if len(update_summary["errors"]) > 5:
            print(f"  ... and {len(update_summary['errors']) - 5} more errors")
    
    return update_summary

In [27]:
# Example usage of the new functions

# Example 1: Update a single JSON file
print("=== EXAMPLE 1: Update single file ===")
updated_data = update_json_with_pdf_metadata(
    file_path="./test_test2pdf.json",
    pdf_extracted_metadata=metadata,
    save_file=True
)

# Example 2: Batch update multiple directories
print("\n=== EXAMPLE 2: Batch update multiple directories ===")
json_directories = [
    f"../data/raw/legal_cases/legal_case_folder{i}/metadata_folder" 
    for i in range(1, 17)
]

# summary = batch_update_json_files_with_pdf_metadata(
#     pdf_extracted_metadata=metadata,
#     json_dirs=json_directories
# )

# Example 3: Update just one specific file
print("\n=== EXAMPLE 3: Update specific file ===")
# summary = batch_update_json_files_with_pdf_metadata(
#     pdf_extracted_metadata=metadata,
#     specific_file="./test_test2pdf.json"
# )

=== EXAMPLE 1: Update single file ===
✅ Updated ./test_test2pdf.json | Court: None

=== EXAMPLE 2: Batch update multiple directories ===

=== EXAMPLE 3: Update specific file ===
